In [ ]:
# Install Python agent with nr-openai-observability enabled
!pip install git+https://github.com/newrelic/newrelic-python-agent@510f747d54df4d4e6a6214eed5f6dcf5dc816621

In [ ]:
# Common environment variables for configuring New Relic Python Agent
# Only run this section if you did not update these values in your newrelic.ini file
# or in your Jupyter environment.
import os

os.environ["NEW_RELIC_APP_NAME"] = "openai-example"

In [ ]:
# Configure New Relic Python Agent
# Only run this section if you did not update your newrelic.ini file.
import os
os.environ["NEW_RELIC_CUSTOM_INSIGHTS_EVENTS_MAX_ATTRIBUTE_VALUE"] = "4095"
# Apply recommended settings
os.environ["NEW_RELIC_APPLICATION_LOGGING_ENABLED"] = "true"
os.environ["NEW_RELIC_APPLICATION_LOGGING_FORWARDING_ENABLED"] = "true"
os.environ["NEW_RELIC_DISTRIBUTED_TRACING_ENABLED"] = "true"

In [ ]:
# In order to send data to New Relic as well as OpenAI or Bedrock, you need a 
# way to store your secrets. Depending on the environment you are running, 
# there are different ways to access to secrets. We will comment on a few
# common ways to access secrets. You may need to adjust some of the code
# for your specific environment.

# If you are running `jupyter notebook` from the command line, you can set
# the environment variables through your command line before launching
# the notebook server. You can then access your environment variables 
# via `os.getenv('...')`.

# If you have stored your secrets in a file, you can leverage the python
# module dotfile to load those values into your environment variables.
#
# from dotenv import load_dotenv
# load_dotenv()

# If you are running your notebook through Google's Colab service, then you
# can store environment variables within Colab and access those variables 
# via an python library.
#
# from google.colab import userdata
# openai_api_key = userdata.get("OPENAI_API_KEY")

In [ ]:
# Install the nr-openai-observability library
!pip install git+https://github.com/newrelic/nr-openai-observability@staging

In [ ]:
# Validate newrelic.ini file
# Note the filepath with prefix `/content` is intended to be run on Google Colab
!newrelic-admin validate-config newrelic.ini

The next block will attempt to connect your code to New Relic. You will
need to have your New Relic license key either in the `newrelic.ini` file 
or set as an environment variable named `NEW_RELIC_LICENSE_KEY`.

In [ ]:
# Initialize New Relic Python Agent
import newrelic.agent

newrelic.agent.initialize("newrelic.ini")
newrelic.agent.register_application(timeout=10)

In [ ]:
# Initialize New Relic additional OpenAI Monitoring
# This step is only needed for the Limited Preview. This instrumentation 
# will be available directly in the New Relic Agent for GA.
#
# For this specific example, this block is *NOT* needed. This example
# is using the OpenAI 1.x APIs and for that instrumentation, only
# the New Relic Python Agent is needed.
#
#from nr_openai_observability import monitor

#monitor.initialization()

In [ ]:
# When not utilizing a web framework, we need a python function that can be 
# decorated so that the New Relic Python Agent will instrument the method call.

@newrelic.agent.background_task()
@newrelic.agent.function_trace(name="openai")
def invoke_openai():
    from openai import OpenAI

    # prompt is credited to https://prompts.chat/#act-as-a-math-teacher
    prompt = """I want you to act as a math teacher. 
    I will provide some mathematical equations or concepts, 
    and it will be your job to explain them in easy-to-understand terms. 
    This could include providing step-by-step instructions for solving a problem, 
    demonstrating various techniques with visuals or suggesting online 
    resources for further study."""
    user_request = (
        'My first request is “I need help understanding how probability works."'
    )
    
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4",
        max_tokens=500,
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": user_request,
            },
        ],
    )
    print(completion.choices[0].message.content)


In [ ]:
# Make a call to an LLM to generate metrics for NewRelic connection
# Invoking OpenAI for this question will take a little while as it is
# running in a synchronous or blocking mode. Just give it a bit of time
# to finish.

import os

import openai

if os.getenv("OPENAI_API_KEY") is None:
    print("You must set your OPENAI_API_KEY environment variable")
else:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    invoke_openai()


In [ ]:
# Allow the New Relic agent to send final messages as part of shutdown
# The agent by default can send data up to a minute later
newrelic.agent.shutdown_agent(60)